# Neural Network Layers and Ops

In this session we will focus on understanding various tensor operations, NN layers and loss functions

# Import Dependencies

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf

In [ ]:
print(tf.__version__)

# 1. Tensors & Tensor Ops

## Constants

In [ ]:
t = tf.constant([[1., 2., 3.], 
                 [4., 5., 6.]])
t

In [ ]:
t.dtype

In [ ]:
t.shape

## Variables

In [ ]:
t = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
t

In [ ]:
t.dtype, t.shape

In [ ]:
t.numpy()

In [ ]:
t[0,0].assign(99)
t

In [ ]:
t[1].assign([10, 20, 30])
t

In [ ]:
# tf.scatter_nd takes an indices tensor, an updates (original tensor).
t.scatter_nd_update(indices=[[0, 0], [1, 1]],
                    updates=[33., 44.])

## Indexing

In [ ]:
t[1:, :]

In [ ]:
t[:, 1:]

In [ ]:
t

## Basic Ops

In [ ]:
t = t + 10
t

In [ ]:
t = t + tf.Variable([[1, 2, 3]], dtype='float32')
t

In [ ]:
tf.square(t)

In [ ]:
np.square(t)

## Sparse Tensors

In [ ]:
t = tf.SparseTensor(indices=[[0, 0], [1, 1], [2, 2]],
                    values=[1., 2., 3.],
                    dense_shape=[3, 3])
t

In [ ]:
tf.sparse.to_dense(t)

In [ ]:
t2 = t * 5
tf.sparse.to_dense(t2)

# 2. Basic Regression with Simple NN + Custom Loss Function

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

housing

In [ ]:
print(housing['DESCR'])

In [ ]:
X = pd.DataFrame(housing['data'], columns=housing['feature_names'])
X.head()

In [ ]:
y = pd.DataFrame({'price': housing['target']})
y.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

In [ ]:
predictions = lr.predict(X_test_scaled)
print('R2:', r2_score(y_test, predictions))
print('MSE:', mean_squared_error(y_test, predictions))

## 2.1: Creating a custom loss function

Create a `mse_loss(...)` function with two arguments: 
- the true labels `y_true` 
- the model predictions `y_pred` 

Make it return the mean squared error using TensorFlow operations. Note that you could write your own custom metrics in this way. 

__Tip:__ Recall that the MSE is the mean of the squares of prediction errors, which are the differences between the predictions and the labels, so you will need to use `tf.reduce_mean()` and `tf.square()` ops.

In [ ]:
def mse_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_pred - y_true))

## 2.2: Create a simple 1-layer NN

Here we leverage the `Sequential` API of `tf.keras` to create a simple NN with 1 hidden layer

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation="relu", 
                          input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(1),
])

In [ ]:
X_train.shape[1]

In [ ]:
model.compile(loss=mse_loss, 
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=['mean_squared_error'])

In [ ]:
model.summary()

## 2.3: Train the model and Test its performance

We will now train our simple NN and test its performance

3200 houses
32

every batch of data passed to the NN will have 32 houses

1 epoch = 3200 // 32 = 100

loss is computed after each batch of 32 houses and error\loss is backpropagated based on gradients in each layer

In [ ]:
model.fit(X_train_scaled, y_train, 
          epochs=30,
          batch_size=32,
          validation_split=0.1)

In [ ]:
predictions = model.predict(X_test_scaled)

In [ ]:
print('R2:', r2_score(y_test, predictions))
print('MSE:', mean_squared_error(y_test, predictions))

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation="relu", 
                          input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(1),
])

In [ ]:
model.compile(loss=mse_loss, 
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=['mean_squared_error'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train_scaled, y_train, 
          epochs=30,
          batch_size=32,
          validation_split=0.1) 

In [ ]:
predictions = model.predict(X_test_scaled)

In [ ]:
print('R2:', r2_score(y_test, predictions))
print('MSE:', mean_squared_error(y_test, predictions))

In [ ]:
z = np.linspace(-5, 5, 200)
plt.figure(figsize=(15,6))
plt.plot(z, np.sign(z), "r-", linewidth=1, label="Step")
plt.plot(z, tf.nn.sigmoid(z), "g--", linewidth=2, label="Sigmoid")
plt.plot(z, tf.nn.tanh(z), "b-", linewidth=2, label="Tanh")
plt.plot(z, tf.nn.relu(z), "m-.", linewidth=2, label="ReLU")
plt.grid(True)
plt.legend(loc="upper left", fontsize=14)
plt.title("Activation functions", fontsize=14)
plt.axis([-5, 5, -1.2, 2]);